# 描画テストプログラム
- 描画例１～４のテストプログラム
- 描画例１：単色で四角形を描画（フレームバッファ全体を指定色で塗りつぶす）
- 描画例２：単色で四角形を描画（フレームバッファの一部を指定色で塗りつぶす）
- 描画例３：単色で四角形を描画（描画領域より広い範囲を描画指定する）
- 描画例４：単色で四角形を描画（フレームバッファをはみ出るよう領域指定して描画する）

### [STEP-1] 定数定義
- 表示解像度パラメータ（VGA or XGA）※描画回路ではVGA固定
- レジスタアドレス設定

In [ ]:
# 表示回路２解像度パラメータ
PIXEL_VGA_WIDTH  = 640
PIXEL_VGA_HEIGHT = 480

PIXEL_XGA_WIDTH  = 1024
PIXEL_XGA_HEIGHT = 768

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

# 描画回路レジスタオフセット
RA_DRAWCTRL    = 0x00002000
RA_DRAWSTAT    = 0x00002004
RA_DRAWBUFSTAT = 0x00002008
RA_DRAWCMD     = 0x0000200C
RA_DRAWINT     = 0x00002010

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [ ]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 描画回路＋表示回路２のコンフィギュレーション

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] DisplayPort初期化
- 表示回路２の解像度を設定
- 描画回路はVGA固定
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)

In [ ]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

PIXEL_WIDTH  = PIXEL_VGA_WIDTH
PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

VGA setting @ DisplayPort 640 480


### [STEP-5] VRAM領域確保
- 高さ x 幅 x 4byte(RGBA)で１フレーム分のデータとなる

In [ ]:
from pynq import allocate

fb0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")

### [STEP-6] 表示回路レジスタ設定
- VBLANK待ち @ DISPCTRL
- Overflow/Underflowフラグクリア @ DISPFIFO 
- VRAMアドレス設定 @ DISPADDR
- 表示ON @ DISPCTRL

In [ ]:
#同期処理（ブランク待ち）
pl.regbus_0.write(RA_DISPCTRL, 0x00000003)
while pl.regbus_0.read(RA_DISPCTRL) != 0x00000003 :
    pass

#Overflow/Underflow フラグクリア
pl.regbus_0.write(RA_DISPFIFO, 0x00000003)

#VRAMアドレス設定
pl.regbus_0.write(RA_DISPADDR, fb0.device_address)

#表示ON
pl.regbus_0.write(RA_DISPCTRL, 0x00000001)

### [STEP-7] 描画例１
- 単色で四角形を描画
- フレームバッファ全体を指定色で塗りつぶす

In [ ]:
#VRAM Clear
fb0.fill(0)

#SETFRAME
pl.regbus_0.write(RA_DRAWCMD, 0x20000000)            #SETFRAME
pl.regbus_0.write(RA_DRAWCMD, fb0.device_address)    #VRAMADR
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x21000000)            #SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x00000000)            #POSX=0, POSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x00FF0000)            #RED

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x00000000)            #DPOSX=0, DPOSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #DSIZEX=640 DSIZEY=480

#EODL
pl.regbus_0.write(RA_DRAWCMD, 0x0F000000)            #EODL

#描画開始&終了待ち
pl.regbus_0.write(RA_DRAWCTRL, 0x00000001)           #EXE
while (pl.regbus_0.read(RA_DRAWSTAT) & 0x00000001) != 0x00000000 :
    pass
pl.regbus_0.write(RA_DRAWCTRL, 0x00000002)           #RST

### [STEP-8] 描画例２
- 単色で四角形を描画
- フレームバッファの一部を指定色で塗りつぶす

In [ ]:
#VRAM Clear
fb0.fill(0)

#SETFRAME
pl.regbus_0.write(RA_DRAWCMD, 0x20000000)            #SETFRAME
pl.regbus_0.write(RA_DRAWCMD, fb0.device_address)    #VRAMADR
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x21000000)            #SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x00000000)            #POSX=0, POSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x0000FF00)            #GREEN

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, (128<<16 | 120))       #DPOSX=128, DPOSY=120
pl.regbus_0.write(RA_DRAWCMD, (320<<16 | 240))       #DSIZEX=320 DSIZEY=240

#EODL
pl.regbus_0.write(RA_DRAWCMD, 0x0F000000)            #EODL

#描画開始&終了待ち
pl.regbus_0.write(RA_DRAWCTRL, 0x00000001)           #EXE
while (pl.regbus_0.read(RA_DRAWSTAT) & 0x00000001) != 0x00000000 :
    pass
pl.regbus_0.write(RA_DRAWCTRL, 0x00000002)           #RST

### [STEP-9] 描画例３
- 単色で四角形を描画
- 描画領域より広い範囲を描画指定する

In [ ]:
#VRAM Clear
fb0.fill(0)

#SETFRAME
pl.regbus_0.write(RA_DRAWCMD, 0x20000000)            #SETFRAME
pl.regbus_0.write(RA_DRAWCMD, fb0.device_address)    #VRAMADR
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x21000000)            #SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, (128<<16 | 120))       #POSX=128, POSY=120
pl.regbus_0.write(RA_DRAWCMD, (320<<16 | 240))       #WIDTH=320 HEIGHT=240

#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x000000FF)            #BLUE

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, (0<<16 | 0))           #DPOSX=0, DPOSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #DSIZEX=640 DSIZEY=480

#EODL
pl.regbus_0.write(RA_DRAWCMD, 0x0F000000)            #EODL

#描画開始&終了待ち
pl.regbus_0.write(RA_DRAWCTRL, 0x00000001)           #EXE
while (pl.regbus_0.read(RA_DRAWSTAT) & 0x00000001) != 0x00000000 :
    pass
pl.regbus_0.write(RA_DRAWCTRL, 0x00000002)           #RST

### [STEP-10] 描画例４
- 単色で四角形を描画
- フレームバッファをはみ出るよう領域指定して描画する

In [ ]:
#VRAM Clear
fb0.fill(0)

#SETFRAME
pl.regbus_0.write(RA_DRAWCMD, 0x20000000)            #SETFRAME
pl.regbus_0.write(RA_DRAWCMD, fb0.device_address)    #VRAMADR
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x21000000)            #SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, (0<<16 | 0))           #POSX=0, POSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x00FFFF00)            #YELLOW

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, (512<<16 | 360))       #DPOSX=512, DPOSY=360
pl.regbus_0.write(RA_DRAWCMD, (320<<16 | 240))       #DSIZEX=320 DSIZEY=240

#EODL
pl.regbus_0.write(RA_DRAWCMD, 0x0F000000)            #EODL

#描画開始&終了待ち
pl.regbus_0.write(RA_DRAWCTRL, 0x00000001)           #EXE
while (pl.regbus_0.read(RA_DRAWSTAT) & 0x00000001) != 0x00000000 :
    pass
pl.regbus_0.write(RA_DRAWCTRL, 0x00000002)           #RST

### [STEP-11] 描画例（ブレンドのテスト）
- 左上に赤の四角を描画
- 中心に青で書いて半透明にして重ねる

In [ ]:
#VRAM Clear
fb0.fill(0)

#SETFRAME
pl.regbus_0.write(RA_DRAWCMD, 0x20000000)            #SETFRAME
pl.regbus_0.write(RA_DRAWCMD, fb0.device_address)    #VRAMADR
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, 0x21000000)            #SETDRAWAREA
pl.regbus_0.write(RA_DRAWCMD, (0<<16 | 0))           #POSX=0, POSY=0
pl.regbus_0.write(RA_DRAWCMD, (640<<16 | 480))       #WIDTH=640 HEIGHT=480

#左上の四角の描画
#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x00FF0000)            #RED

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, (0<<16 | 0))           #DPOSX=0, DPOSY=0
pl.regbus_0.write(RA_DRAWCMD, (320<<16 | 240))       #DSIZEX=320 DSIZEY=240

#中央の四角の描画
#SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x23000000)            #SETFCOLOR
pl.regbus_0.write(RA_DRAWCMD, 0x000000FF)            #BLUE

#SETBLENDALPHA
pl.regbus_0.write(RA_DRAWCMD, 0x33034E00)            #SETBLENDALPHA, A=0, B=1, C=5, D=1, E=6, SRCCA=0
pl.regbus_0.write(RA_DRAWCMD, 0x80808080)            #COEF0
pl.regbus_0.write(RA_DRAWCMD, 0xFFFFFFFF)            #COEF1

#PATBLT
pl.regbus_0.write(RA_DRAWCMD, 0x81000000)            #PATBLT
pl.regbus_0.write(RA_DRAWCMD, (128<<16 | 120))       #DPOSX=128, DPOSY=120
pl.regbus_0.write(RA_DRAWCMD, (320<<16 | 240))       #DSIZEX=320 DSIZEY=240

#EODL
pl.regbus_0.write(RA_DRAWCMD, 0x0F000000)            #EODL

#描画開始&終了待ち
pl.regbus_0.write(RA_DRAWCTRL, 0x00000001)           #EXE
while (pl.regbus_0.read(RA_DRAWSTAT) & 0x00000001) != 0x00000000 :
    pass
pl.regbus_0.write(RA_DRAWCTRL, 0x00000002)           #RST

### [STEP-12] 表示OFF＆PYNQの初期画面に戻す
- 表示OFF @ DISPCTRL
- DisplayPortの制御は、PL側からPS側に切り替わります。

In [ ]:
#リリース処理
fb0.freebuffer()

#表示OFF設定
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)
lvdp.close()